In [35]:
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import datetime
import time
import pickle
import importlib
%matplotlib inline


In [2]:
from finrl.apps import config
from finrl.neo_finrl.preprocessor.yahoodownloader import YahooDownloader
from finrl.neo_finrl.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.neo_finrl.env_stock_trading.env_stocktrading import StockTradingEnv
# from finrl.neo_finrl.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint
import sys
import itertools

/home/sirius/.local/lib/python3.9/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [4]:
print(config.START_DATE)
if not os.path.exists('./'+ config.DATA_SAVE_DIR + '/DF.pkl' ):
    # os.makedirs('./' + config.DATA_SAVE_DIR + '/DF.pkl):
    df = YahooDownloader(start_date = '2009-01-01',
                         end_date = config.END_DATE, 
                         ticker_list = config.DOW_30_TICKER).fetch_data()
    with open('./' + config.DATA_SAVE_DIR + '/DF.pkl', 'wb') as file:
        pickle.dump(df, file)
else:
    with open('./' + config.DATA_SAVE_DIR + '/DF.pkl', 'rb') as file:
        df = pickle.load(file)

2009-01-01


In [5]:
# df[df['tic'] == 'DOW'].head(50)

In [6]:
# df.shape
#df.sort_values(['date','tic'],ignore_index=True).head(40)
fe = FeatureEngineer(use_technical_indicator=False, 
                     tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                     use_vix = False, 
                     use_turbulence = False, 
                     user_defined_feature = False)
processed = fe.preprocess_data(df)

In [7]:
list_ticker = processed['tic'].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype((str)))
combination = list(itertools.product(list_date,list_ticker)) 
processed_full = pd.DataFrame(combination, columns=['date','tic']).merge(processed, on=['date','tic'],how='left')
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])
processed_full = processed_full.fillna(0)

# print(processed_full.sort_values(['date','tic'], ignore_index = True).head(10))
# print(type(processed_full))


In [8]:
processed_full.sort_values(['date','tic'], ignore_index = True).head(10)

,date,tic,open,high,low,close,volume,day
0,2008-12-31,AAPL,3.070357,3.133571,3.047857,2.613432,607541200.0,2.0
1,2008-12-31,AMGN,57.110001,58.220001,57.060001,44.657005,6287200.0,2.0
2,2008-12-31,AXP,17.969999,18.750000,17.910000,14.988311,9625600.0,2.0
3,2008-12-31,BA,41.590000,43.049999,41.500000,32.005890,5443100.0,2.0
4,2008-12-31,CAT,43.700001,45.099998,43.700001,30.925045,6277400.0,2.0
5,2008-12-31,CRM,7.712500,8.130000,7.707500,8.002500,5367600.0,2.0
6,2008-12-31,CSCO,16.180000,16.549999,16.120001,11.938444,37513700.0,2.0
7,2008-12-31,CVX,72.900002,74.629997,72.900002,44.129261,9964300.0,2.0
8,2008-12-31,DIS,22.570000,22.950001,22.520000,19.538342,9012100.0,2.0
9,2008-12-31,GS,82.239998,86.150002,81.120003,69.999687,14894100.0,2.0


In [9]:
train = data_split(processed_full, '2009-01-01', '2020-07-01')
trade = data_split(processed_full, '2020-07-01', '2021-10-31')
print(len(train), type(train))
print(len(trade))

83897 <class 'pandas.core.frame.DataFrame'>
9773


In [10]:
# train.tail()

In [11]:
# trade.head()

In [12]:
# config.TECHNICAL_INDICATORS_LIST
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension
print(f'Stock Dimension: {stock_dimension}, State Space: {state_space}')

Stock Dimension: 29, State Space: 59


In [13]:
# print(config.DOW_30_TICKER)
# print(train.tic.unique())
# [x for x in config.DOW_30_TICKER if x not in train.tic.unique()]

In [14]:
env_kwargs = {
    'hmax':100,
    'initial_amount': 1000000,
    'buy_cost_pct'  : 0.001, 
    'sell_cost_pct' : 0.001,
    'state_space'   : state_space, 
    'stock_dim'     : stock_dimension, 
    'tech_indicator_list' : [],
    'action_space'        : stock_dimension, 
    'reward_scaling'      : 1e-4}
e_train_gym = StockTradingEnv(df = train, **env_kwargs)
env_train, e = e_train_gym.get_sb_env()
type(e_train_gym), type(env_train)
env_train

In [43]:
import Utils
importlib.reload(Utils)
agent = Utils.Agent(env = env_train)


In [44]:
TD3_PARAMS = {'batch_size': 1,
              'buffer_size': int(1e6),
              'learning_rate': 1e-3}
PPO_PARAMS = {'n_steps':2048,
              'ent_coef':1e-2,
              'learning_rate': 0.00025,
              'batch_size':128}

agent.specify_model('ppo', model_kwargs = PPO_PARAMS)
# agent.specify_model('td3', model_kwargs=TD3_PARAMS)

Using cpu device


In [45]:
t1 = time.time()
agent.train(tb_log_name='ppo', total_timesteps=1)
print(time.time() - t1)

-----------------------------
| time/              |      |
|    fps             | 28   |
|    iterations      | 1    |
|    time_elapsed    | 72   |
|    total_timesteps | 2048 |
-----------------------------
74.68159890174866
